# Step 5: LLM Orchestrator

This notebook implements the LLM Orchestrator for the accounting automation pipeline.

**Input:**
- Structured JSON from data extraction (step 2)
- Embedding vectors (.npy or list of floats) from step 3
- Reconciled similarity matches and deduplicated entries from step 4

**Output:**
- Reasoning trace
- Validation logs
- Explanations

**Model:** Gemini 2.5 Pro via LangChain


In [ ]:
# Install required packages
!pip install -q langchain langchain-google-genai google-generativeai fastapi uvicorn pydantic numpy pandas python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-community
7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.0.10 requires langchain-core<0.4.0,>=0.3.37, but you have langchain-core 1.0.4 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-s

In [ ]:
# Import libraries
import os
import json
import numpy as np
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
from pathlib import Path
import logging
from dataclasses import dataclass, asdict
from enum import Enum

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackHandler

# Pydantic for validation
from pydantic import BaseModel, Field, validator

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Configuration

Set your Google API key here. You can get it from [Google AI Studio](https://makersuite.google.com/app/apikey)


In [ ]:
# Configuration
from google.colab import userdata

# 🔑 Replace with your actual API key
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
# Model configuration
# Try Gemini 2.5 Pro first. If not available, fallback to: "gemini-1.5-pro" or "gemini-2.0-flash-exp"
MODEL_NAME = "gemini-2.5-pro"  # Gemini 2.5 Pro model
TEMPERATURE = 0.1  # Low temperature for consistent, factual outputs
MAX_TOKENS = 4096

# Initialize the LLM
try:
    llm = ChatGoogleGenerativeAI(
        model=MODEL_NAME,
        google_api_key=GOOGLE_API_KEY,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        convert_system_message_to_human=True
    )
    print(f"Initialized LLM with model: {MODEL_NAME}")
except Exception as e:
    print(f"Error initializing {MODEL_NAME}: {e}")
    print("Falling back to gemini-1.5-pro...")
    MODEL_NAME = "gemini-1.5-pro"
    llm = ChatGoogleGenerativeAI(
        model=MODEL_NAME,
        google_api_key=GOOGLE_API_KEY,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        convert_system_message_to_human=True
    )
    print(f"Initialized LLM with fallback model: {MODEL_NAME}")


Initialized LLM with model: gemini-2.5-pro


## Data Models


In [ ]:
# Data models for structured input/output
class ValidationStatus(str, Enum):
    VALID = "valid"
    INVALID = "invalid"
    WARNING = "warning"
    NEEDS_REVIEW = "needs_review"

class ExtractedRecord(BaseModel):
    """Structured JSON record from step 2"""
    vendor: Optional[str] = None
    date: Optional[str] = None
    amount: Optional[float] = None
    tax: Optional[float] = None
    total: Optional[float] = None
    invoice_number: Optional[str] = None
    description: Optional[str] = None
    category: Optional[str] = None
    payment_method: Optional[str] = None
    raw_text: Optional[str] = None

class ReconciliationMatch(BaseModel):
    """Reconciled similarity match from step 4"""
    record_id: str
    matched_record_ids: List[str]
    similarity_scores: List[float]
    is_duplicate: bool
    confidence: float

class ValidationResult(BaseModel):
    """Validation result for a record"""
    record_id: str
    status: ValidationStatus
    issues: List[str]
    confidence: float
    reasoning: str

class ReasoningTrace(BaseModel):
    """Reasoning trace for a record"""
    record_id: str
    timestamp: str
    steps: List[Dict[str, Any]]
    final_conclusion: str
    confidence_score: float

class OrchestrationOutput(BaseModel):
    """Complete orchestration output"""
    record_id: str
    validation_result: ValidationResult
    reasoning_trace: ReasoningTrace
    explanation: str
    recommendations: List[str]
    metadata: Dict[str, Any]


## LLM Orchestrator Class


In [ ]:
class LLMOrchestrator:
    """
    LLM Orchestrator for accounting automation pipeline.
    Handles validation, reasoning, and explanation generation.
    """

    def __init__(self, llm: ChatGoogleGenerativeAI):
        self.llm = llm
        self.validation_logs = []
        self.reasoning_traces = []

        # Define prompts
        self.validation_prompt = self._create_validation_prompt()
        self.reasoning_prompt = self._create_reasoning_prompt()
        self.explanation_prompt = self._create_explanation_prompt()

    def _create_validation_prompt(self) -> ChatPromptTemplate:
        """Create prompt for validation"""
        template = """You are an expert accounting validation system. Your task is to validate extracted financial records.

Given a structured JSON record, analyze it for:
1. Completeness: Are all critical fields present?
2. Consistency: Do the numbers add up correctly (amount + tax = total)?
3. Format: Are dates, amounts, and other fields in correct format?
4. Reasonableness: Are the values within expected ranges?
5. Business logic: Does the record make business sense?

Extracted Record:
{record_json}

Reconciliation Information:
{reconciliation_info}

Provide your validation in the following JSON format:
{{
    "status": "valid|invalid|warning|needs_review",
    "issues": ["list of issues found"],
    "confidence": 0.0-1.0,
    "reasoning": "detailed explanation of validation"
}}

Be thorough and precise. Flag any potential issues."""

        return ChatPromptTemplate.from_messages([
            SystemMessage(content="You are an expert accounting validation system."),
            HumanMessage(content=template)
        ])

    def _create_reasoning_prompt(self) -> ChatPromptTemplate:
        """Create prompt for reasoning trace"""
        template = """You are an AI reasoning system for accounting records. Generate a step-by-step reasoning trace.

Record:
{record_json}

Reconciliation Info:
{reconciliation_info}

Validation Result:
{validation_result}

Generate a detailed reasoning trace showing:
1. Initial analysis of the record
2. Field-by-field examination
3. Cross-field consistency checks
4. Reconciliation analysis (if applicable)
5. Final conclusion

Format as JSON:
{{
    "steps": [
        {{"step": 1, "action": "analyzed field X", "observation": "...", "conclusion": "..."}},
        ...
    ],
    "final_conclusion": "summary of reasoning",
    "confidence_score": 0.0-1.0
}}"""

        return ChatPromptTemplate.from_messages([
            SystemMessage(content="You are an AI reasoning system for accounting records."),
            HumanMessage(content=template)
        ])

    def _create_explanation_prompt(self) -> ChatPromptTemplate:
        """Create prompt for explanation generation"""
        template = """Generate a clear, human-readable explanation of the validation and reasoning process.

Record:
{record_json}

Validation Result:
{validation_result}

Reasoning Trace:
{reasoning_trace}

Provide:
1. A summary of what was analyzed
2. Key findings
3. Any issues or concerns
4. Recommendations for next steps

Write in clear, professional language suitable for accounting professionals."""

        return ChatPromptTemplate.from_messages([
            SystemMessage(content="You are an expert accounting analyst explaining validation results."),
            HumanMessage(content=template)
        ])

    def validate_record(
        self,
        record: Dict[str, Any],
        reconciliation_info: Optional[Dict[str, Any]] = None
    ) -> ValidationResult:
        """Validate a single record using LLM"""
        try:
            # Prepare inputs
            record_json = json.dumps(record, indent=2)
            recon_json = json.dumps(reconciliation_info, indent=2) if reconciliation_info else "No reconciliation data"

            # Create prompt
            messages = self.validation_prompt.format_messages(
                record_json=record_json,
                reconciliation_info=recon_json
            )

            # Call LLM
            response = self.llm.invoke(messages)
            response_text = response.content

            # Parse JSON from response (handle markdown code blocks)
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0].strip()

            validation_data = json.loads(response_text)

            # Create validation result
            result = ValidationResult(
                record_id=record.get("record_id", "unknown"),
                status=ValidationStatus(validation_data.get("status", "needs_review")),
                issues=validation_data.get("issues", []),
                confidence=validation_data.get("confidence", 0.5),
                reasoning=validation_data.get("reasoning", "")
            )

            # Log validation
            self.validation_logs.append({
                "timestamp": datetime.now().isoformat(),
                "record_id": result.record_id,
                "status": result.status.value,
                "issues": result.issues
            })

            return result

        except Exception as e:
            logger.error(f"Validation error: {e}")
            return ValidationResult(
                record_id=record.get("record_id", "unknown"),
                status=ValidationStatus.NEEDS_REVIEW,
                issues=[f"Validation error: {str(e)}"],
                confidence=0.0,
                reasoning=f"Error during validation: {str(e)}"
            )

    def generate_reasoning_trace(
        self,
        record: Dict[str, Any],
        validation_result: ValidationResult,
        reconciliation_info: Optional[Dict[str, Any]] = None
    ) -> ReasoningTrace:
        """Generate reasoning trace for a record"""
        try:
            record_json = json.dumps(record, indent=2)
            recon_json = json.dumps(reconciliation_info, indent=2) if reconciliation_info else "No reconciliation data"
            validation_json = json.dumps(asdict(validation_result), indent=2)

            messages = self.reasoning_prompt.format_messages(
                record_json=record_json,
                reconciliation_info=recon_json,
                validation_result=validation_json
            )

            response = self.llm.invoke(messages)
            response_text = response.content

            # Parse JSON from response
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0].strip()

            trace_data = json.loads(response_text)

            trace = ReasoningTrace(
                record_id=record.get("record_id", "unknown"),
                timestamp=datetime.now().isoformat(),
                steps=trace_data.get("steps", []),
                final_conclusion=trace_data.get("final_conclusion", ""),
                confidence_score=trace_data.get("confidence_score", 0.5)
            )

            self.reasoning_traces.append(asdict(trace))

            return trace

        except Exception as e:
            logger.error(f"Reasoning trace error: {e}")
            return ReasoningTrace(
                record_id=record.get("record_id", "unknown"),
                timestamp=datetime.now().isoformat(),
                steps=[{"step": 1, "action": "error", "observation": str(e), "conclusion": "Error occurred"}],
                final_conclusion=f"Error generating reasoning trace: {str(e)}",
                confidence_score=0.0
            )

    def generate_explanation(
        self,
        record: Dict[str, Any],
        validation_result: ValidationResult,
        reasoning_trace: ReasoningTrace
    ) -> str:
        """Generate human-readable explanation"""
        try:
            record_json = json.dumps(record, indent=2)
            validation_json = json.dumps(asdict(validation_result), indent=2)
            trace_json = json.dumps(asdict(reasoning_trace), indent=2)

            messages = self.explanation_prompt.format_messages(
                record_json=record_json,
                validation_result=validation_json,
                reasoning_trace=trace_json
            )

            response = self.llm.invoke(messages)
            return response.content

        except Exception as e:
            logger.error(f"Explanation generation error: {e}")
            return f"Error generating explanation: {str(e)}"

    def orchestrate(
        self,
        structured_json: Dict[str, Any],
        embedding_vector: Optional[np.ndarray] = None,
        reconciliation_data: Optional[Dict[str, Any]] = None
    ) -> OrchestrationOutput:
        """
        Main orchestration method that combines all steps.

        Args:
            structured_json: Extracted JSON record from step 2
            embedding_vector: Embedding vector from step 3 (optional, for metadata)
            reconciliation_data: Reconciliation matches from step 4 (optional)

        Returns:
            Complete orchestration output with validation, reasoning, and explanation
        """
        record_id = structured_json.get("record_id", f"record_{datetime.now().timestamp()}")
        structured_json["record_id"] = record_id

        # Step 1: Validate
        logger.info(f"Validating record {record_id}...")
        validation_result = self.validate_record(structured_json, reconciliation_data)

        # Step 2: Generate reasoning trace
        logger.info(f"Generating reasoning trace for record {record_id}...")
        reasoning_trace = self.generate_reasoning_trace(
            structured_json,
            validation_result,
            reconciliation_data
        )

        # Step 3: Generate explanation
        logger.info(f"Generating explanation for record {record_id}...")
        explanation = self.generate_explanation(structured_json, validation_result, reasoning_trace)

        # Step 4: Generate recommendations
        recommendations = self._generate_recommendations(validation_result, reasoning_trace)

        # Step 5: Prepare metadata
        # Handle embedding shape for both numpy arrays and lists
        embedding_shape = None
        if embedding_vector is not None:
            if isinstance(embedding_vector, np.ndarray):
                embedding_shape = list(embedding_vector.shape)
            elif isinstance(embedding_vector, list):
                embedding_shape = [len(embedding_vector)]
            else:
                embedding_shape = "unknown"

        metadata = {
            "embedding_shape": embedding_shape,
            "has_reconciliation": reconciliation_data is not None,
            "processing_timestamp": datetime.now().isoformat()
        }

        return OrchestrationOutput(
            record_id=record_id,
            validation_result=validation_result,
            reasoning_trace=reasoning_trace,
            explanation=explanation,
            recommendations=recommendations,
            metadata=metadata
        )

    def _generate_recommendations(
        self,
        validation_result: ValidationResult,
        reasoning_trace: ReasoningTrace
    ) -> List[str]:
        """Generate recommendations based on validation and reasoning"""
        recommendations = []

        if validation_result.status == ValidationStatus.INVALID:
            recommendations.append("Record requires manual review before processing")
            recommendations.append("Fix identified issues before ledger entry")

        if validation_result.status == ValidationStatus.WARNING:
            recommendations.append("Review flagged issues before finalizing")

        if validation_result.confidence < 0.7:
            recommendations.append("Low confidence score - consider manual verification")

        if validation_result.issues:
            recommendations.append(f"Address {len(validation_result.issues)} identified issue(s)")

        if not recommendations:
            recommendations.append("Record appears valid and ready for ledger entry")

        return recommendations

    def get_validation_logs(self) -> List[Dict[str, Any]]:
        """Get all validation logs"""
        return self.validation_logs

    def get_reasoning_traces(self) -> List[Dict[str, Any]]:
        """Get all reasoning traces"""
        return self.reasoning_traces

    def export_logs(self, filepath: str):
        """Export all logs to JSON file"""
        export_data = {
            "validation_logs": self.validation_logs,
            "reasoning_traces": self.reasoning_traces,
            "export_timestamp": datetime.now().isoformat()
        }
        with open(filepath, 'w') as f:
            json.dump(export_data, f, indent=2)
        logger.info(f"Logs exported to {filepath}")


In [ ]:
# Initialize the orchestrator
orchestrator = LLMOrchestrator(llm)
print("LLM Orchestrator initialized successfully!")


LLM Orchestrator initialized successfully!


## Example Usage

### Example 1: Process a single record with all inputs


In [ ]:
# Example structured JSON from step 2
example_record = {
    "vendor": "Amazon Web Services",
    "date": "2024-01-15",
    "amount": 1250.00,
    "tax": 112.50,
    "total": 1362.50,
    "invoice_number": "INV-2024-001234",
    "description": "Cloud hosting services - January 2024",
    "category": "IT Services",
    "payment_method": "Credit Card",
    "raw_text": "Invoice from AWS for cloud services..."
}

# Example embedding vector from step 3 (dummy data for demonstration)
example_embedding = np.random.rand(768)  # Typical embedding dimension

# Example reconciliation data from step 4
example_reconciliation = {
    "matched_record_ids": ["record_001", "record_002"],
    "similarity_scores": [0.95, 0.87],
    "is_duplicate": False,
    "confidence": 0.92
}

# Process the record
print("Processing record through LLM Orchestrator...")
print("=" * 80)

output = orchestrator.orchestrate(
    structured_json=example_record,
    embedding_vector=example_embedding,
    reconciliation_data=example_reconciliation
)

# Display results
print("\n" + "=" * 80)
print("ORCHESTRATION OUTPUT")
print("=" * 80)
print(f"\nRecord ID: {output.record_id}")
print(f"\nValidation Status: {output.validation_result.status.value}")
print(f"Confidence: {output.validation_result.confidence:.2f}")
print(f"Issues: {output.validation_result.issues}")
print(f"\nReasoning: {output.validation_result.reasoning}")

print("\n" + "-" * 80)
print("REASONING TRACE")
print("-" * 80)
for i, step in enumerate(output.reasoning_trace.steps, 1):
    print(f"\nStep {i}:")
    print(f"  Action: {step.get('action', 'N/A')}")
    print(f"  Observation: {step.get('observation', 'N/A')}")
    print(f"  Conclusion: {step.get('conclusion', 'N/A')}")

print("\n" + "-" * 80)
print("EXPLANATION")
print("-" * 80)
print(output.explanation)

print("\n" + "-" * 80)
print("RECOMMENDATIONS")
print("-" * 80)
for i, rec in enumerate(output.recommendations, 1):
    print(f"{i}. {rec}")

print("\n" + "-" * 80)
print("METADATA")
print("-" * 80)
print(json.dumps(output.metadata, indent=2))


Processing record through LLM Orchestrator...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
ERROR:__main__:Reasoning trace error: asdict() should be called on dataclass instances
ERROR:__main__:Explanation generation error: asdict() should be called on dataclass instances



ORCHESTRATION OUTPUT

Record ID: record_1762686473.403411

Validation Status: invalid
Confidence: 1.00
Issues: ["Format Error: 'invoice_date' (2023-13-01) is an invalid date. The month '13' does not exist.", "Format Error: 'due_date' (2023-02-30) is an invalid date. February does not have 30 days.", 'Consistency Error: The sum of amount (1500.00) and tax (120.00) is 1620.00, which does not match the provided total (1600.00).', 'Reconciliation Mismatch: The invoice total (1600.00) does not match the Purchase Order total (1620.00).']

Reasoning: The record is invalid due to multiple critical errors in format, consistency, and reconciliation.

1.  **Completeness:** All critical fields (invoice_id, vendor_name, dates, amounts) are present. No issues found.

2.  **Format:** The record contains two invalid dates. The 'invoice_date' of '2023-13-01' is impossible as there is no 13th month. The 'due_date' of '2023-02-30' is also impossible as February 2023 only has 28 days. All numeric and str

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

# 🔑 Replace with your actual API key
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")  # Set your API key here or use environment variable

# Configure Gemini client
genai.configure(api_key=GOOGLE_API_KEY)

# ✅ STEP 3: Initialize and test connection
try:
    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content("Hello Gemini! Can you confirm the connection is working?")
    print("✅ Gemini Connection Successful!\n")
    print("Response:", response.text)

except Exception as e:
    print("❌ Connection failed.\n")
    print("Error:", str(e))

✅ Gemini Connection Successful!

Response: Hello! Yes, I'm here and ready to chat. How can I help you today?


### Example 2: Process multiple records in batch


In [ ]:
# Example batch of records
batch_records = [
    {
        "vendor": "Office Supplies Co",
        "date": "2024-01-20",
        "amount": 450.00,
        "tax": 40.50,
        "total": 490.50,
        "invoice_number": "OS-2024-0056",
        "description": "Office supplies and stationery",
        "category": "Office Expenses"
    },
    {
        "vendor": "Tech Solutions Inc",
        "date": "2024-01-22",
        "amount": 5000.00,
        "tax": 500.00,
        "total": 5500.00,
        "invoice_number": "TS-2024-0123",
        "description": "Software licenses - Annual",
        "category": "Software"
    },
    {
        "vendor": "Utilities Corp",
        "date": "2024-01-25",
        "amount": 850.00,
        "tax": 76.50,
        "total": 926.50,
        "invoice_number": "UT-2024-0789",
        "description": "Electricity bill - January",
        "category": "Utilities"
    }
]

# Process batch
print("Processing batch of records...")
print("=" * 80)

batch_outputs = []
for i, record in enumerate(batch_records, 1):
    print(f"\nProcessing record {i}/{len(batch_records)}...")
    output = orchestrator.orchestrate(
        structured_json=record,
        embedding_vector=np.random.rand(768),  # Dummy embedding
        reconciliation_data=None  # No reconciliation for this example
    )
    batch_outputs.append(output)
    print(f"  Status: {output.validation_result.status.value}")
    print(f"  Confidence: {output.validation_result.confidence:.2f}")

print("\n" + "=" * 80)
print("BATCH PROCESSING SUMMARY")
print("=" * 80)
print(f"Total records processed: {len(batch_outputs)}")
print(f"Valid records: {sum(1 for o in batch_outputs if o.validation_result.status == ValidationStatus.VALID)}")
print(f"Records needing review: {sum(1 for o in batch_outputs if o.validation_result.status == ValidationStatus.NEEDS_REVIEW)}")
print(f"Invalid records: {sum(1 for o in batch_outputs if o.validation_result.status == ValidationStatus.INVALID)}")
print(f"Average confidence: {np.mean([o.validation_result.confidence for o in batch_outputs]):.2f}")


Processing batch of records...

Processing record 1/3...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
ERROR:__main__:Reasoning trace error: asdict() should be called on dataclass instances
ERROR:__main__:Explanation generation error: asdict() should be called on dataclass instances
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


  Status: invalid
  Confidence: 0.10

Processing record 2/3...


ERROR:__main__:Reasoning trace error: asdict() should be called on dataclass instances
ERROR:__main__:Explanation generation error: asdict() should be called on dataclass instances
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


  Status: invalid
  Confidence: 0.98

Processing record 3/3...


ERROR:__main__:Reasoning trace error: asdict() should be called on dataclass instances
ERROR:__main__:Explanation generation error: asdict() should be called on dataclass instances


  Status: invalid
  Confidence: 0.30

BATCH PROCESSING SUMMARY
Total records processed: 3
Valid records: 0
Records needing review: 0
Invalid records: 3
Average confidence: 0.46


### Example 3: Load data from previous steps (JSON and .npy files)


In [ ]:
def load_structured_json(filepath: str) -> Dict[str, Any]:
    """Load structured JSON from step 2"""
    with open(filepath, 'r') as f:
        return json.load(f)

def load_embedding_vector(filepath: str) -> np.ndarray:
    """Load embedding vector from step 3 (.npy file)"""
    return np.load(filepath)

def load_reconciliation_data(filepath: str) -> Dict[str, Any]:
    """Load reconciliation data from step 4"""
    with open(filepath, 'r') as f:
        return json.load(f)

# Example usage (uncomment and modify paths as needed):
# structured_json = load_structured_json("path/to/step2_output.json")
# embedding_vector = load_embedding_vector("path/to/step3_output.npy")
# reconciliation_data = load_reconciliation_data("path/to/step4_output.json")
#
# output = orchestrator.orchestrate(
#     structured_json=structured_json,
#     embedding_vector=embedding_vector,
#     reconciliation_data=reconciliation_data
# )


## Export Logs and Results


In [ ]:
# Export all validation logs and reasoning traces
log_filepath = "orchestrator_logs.json"
orchestrator.export_logs(log_filepath)
print(f"Logs exported to {log_filepath}")

# Display summary of logs
print("\n" + "=" * 80)
print("VALIDATION LOGS SUMMARY")
print("=" * 80)
for log in orchestrator.get_validation_logs():
    print(f"\n{log['timestamp']} - Record: {log['record_id']}")
    print(f"  Status: {log['status']}")
    print(f"  Issues: {len(log['issues'])}")


Logs exported to orchestrator_logs.json

VALIDATION LOGS SUMMARY

2025-11-09T11:08:20.810565 - Record: record_1762686473.403411
  Status: invalid
  Issues: 4

2025-11-09T11:09:18.832774 - Record: record_1762686519.50255
  Status: invalid
  Issues: 3

2025-11-09T11:09:52.111574 - Record: record_1762686558.835192
  Status: invalid
  Issues: 3

2025-11-09T11:10:23.628149 - Record: record_1762686592.113273
  Status: invalid
  Issues: 5


## Save Orchestration Output

Save the complete orchestration output for use in subsequent steps (e.g., ledger engine)


In [ ]:
def save_orchestration_output(output: OrchestrationOutput, filepath: str):
    """Save orchestration output to JSON file"""
    output_dict = {
        "record_id": output.record_id,
        "validation_result": asdict(output.validation_result),
        "reasoning_trace": asdict(output.reasoning_trace),
        "explanation": output.explanation,
        "recommendations": output.recommendations,
        "metadata": output.metadata
    }
    with open(filepath, 'w') as f:
        json.dump(output_dict, f, indent=2)
    print(f"Orchestration output saved to {filepath}")

# Example: Save the output from previous examples
# save_orchestration_output(output, "orchestration_output.json")


## Integration Helper Functions

Helper functions to integrate with the full pipeline


In [ ]:
def process_pipeline_outputs(
    structured_json_path: str,
    embedding_vector_path: Optional[str] = None,
    reconciliation_data_path: Optional[str] = None
) -> OrchestrationOutput:
    """
    Complete pipeline integration function.
    Loads outputs from steps 2, 3, and 4 and processes through orchestrator.

    Args:
        structured_json_path: Path to JSON from step 2
        embedding_vector_path: Path to .npy file from step 3 (optional)
        reconciliation_data_path: Path to JSON from step 4 (optional)

    Returns:
        OrchestrationOutput with all validation, reasoning, and explanations
    """
    # Load inputs
    structured_json = load_structured_json(structured_json_path)

    embedding_vector = None
    if embedding_vector_path and os.path.exists(embedding_vector_path):
        embedding_vector = load_embedding_vector(embedding_vector_path)

    reconciliation_data = None
    if reconciliation_data_path and os.path.exists(reconciliation_data_path):
        reconciliation_data = load_reconciliation_data(reconciliation_data_path)

    # Process through orchestrator
    output = orchestrator.orchestrate(
        structured_json=structured_json,
        embedding_vector=embedding_vector,
        reconciliation_data=reconciliation_data
    )

    return output

# Example usage:
# output = process_pipeline_outputs(
#     structured_json_path="step2_output.json",
#     embedding_vector_path="step3_output.npy",
#     reconciliation_data_path="step4_output.json"
# )


## Notes

1. **API Key Setup**: Make sure to set your `GOOGLE_API_KEY` in the configuration cell above. Get it from [Google AI Studio](https://makersuite.google.com/app/apikey)
2. **Model Selection**: Using `gemini-2.5-pro`. If not available, the code will automatically fallback to `gemini-1.5-pro`
3. **Error Handling**: The orchestrator includes comprehensive error handling for API calls and JSON parsing
4. **Output Format**: All outputs are structured as Pydantic models for easy integration with step 7 (Ledger Engine)
5. **Logging**: All validation logs and reasoning traces are stored and can be exported for audit purposes
6. **Input Formats**:
   - Structured JSON: Dictionary with fields like vendor, date, amount, tax, total, etc.
   - Embedding vectors: NumPy arrays (.npy files) or list of floats
   - Reconciliation data: Dictionary with matched_record_ids, similarity_scores, is_duplicate, confidence
